In [3]:
# ===============================================================================
# 단속건수
# ===============================================================================
import os
import json
import folium
import pandas as pd
import math
from shapely.geometry import shape, Point

os.chdir('C:\\Users\\gkstm\\Desktop\\PIPA_data\\시그니쳐테이블')
# data = pd.read_excel('signiture_table.xlsx', encoding = 'cp949')
data = pd.read_csv('signiture_table.csv', encoding = 'cp949')
# data = pd.read_csv('signature_table.csv', encoding = 'cp949')

values = []
for i in range(len(data)):
    values.append(0)

C:\Anaconda3\lib\site-packages\pandas\compat\_optional.py:106: UserWarning: Pandas requires version '1.2.1' or newer of 'bottleneck' (version '1.1.0' currently installed).
  warnings.warn(msg, UserWarning)


FileNotFoundError: [Errno 2] File b'signiture_table.csv' does not exist: b'signiture_table.csv'

In [6]:
# ===============================================================================
# 함수들
# ===============================================================================
def return_circle(point_origin, r_meter = 200, sides = 20):
    # r은 미터단위 (반지름)
    
    lon_per_meter = 1.10035 * 10e-6
    
    r = r_meter * lon_per_meter
    
    
    coordinates = []
    
    for i in range(0,sides):
        
        seta = i * 360 / sides
        radian = math.radians(seta)
        
        lat = math.sin(radian) * r * (24.668/30.887) + point_origin[0]
        lon = math.cos(radian) * r + point_origin[1]
        
        coordinates.append([lon,lat])
    
    return coordinates


def set_polygons(r_meter = 200):

    polygons = []
    geometrys = []

    for i in range(len(data)):

        lat = data['parking_lat'][i]
        lon = data['parking_lng'][i]

        geo = {}

        geo['coordinates'] = [[]]
        geo['type'] = 'MultiPolygon'
        geo['coordinates'][0].append(return_circle([lat,lon], r_meter = r_meter))

        polygon = shape(geo)

        polygons.append(polygon)
        geometrys.append(geo)
    
    return polygons, geometrys

In [11]:
lat = [35.2000249,35.186545]
lng = [128.7105582,128.7084551]
name = ['갈전운동장 공영주차장','천선동 쓰레기 매립장 입구 임시공영주차장']

data = pd.DataFrame()
data['parking_lat'] = lat
data['parking_lng'] = lng
data['parking_name'] = name

values = []
for i in range(len(data)):
    values.append(0)

In [13]:
polygons = set_polygons()[0]

illegal = pd.read_excel('단속건수.xlsx',encoding = 'cp949')

for i in range(len(values)):
    values[i] = 0

for i in range(len(illegal)):
    point = Point(illegal['lon'][i],illegal['lat'][i])
    
    for num in range(len(data)):
        polygon = polygons[num]
        if(polygon.contains(point)):
            values[num] += illegal['단속건수'][i]

data['D'] = values

# data.to_csv('signature_table.csv', encoding = 'cp949')


illegal = pd.read_csv('cctv.csv',encoding = 'cp949')

for i in range(len(values)):
    values[i] = 0

for i in range(len(illegal)):
    point = Point(illegal['lon'][i],illegal['lat'][i])
    
    for num in range(len(data)):
        polygon = polygons[num]
        if(polygon.contains(point)):
            values[num] += 1

data['C'] = values

data.to_csv('signature_table.csv', encoding = 'cp949')




months = [
    '01','02','03','04','05','06','07','08','09','10','11','12'
]

weekday = ['MON','TUS','WED','THU','FRI','SAT','SUN']



flow_table = pd.DataFrame()
flow_table['parking_name'] = data['parking_name']
flow_table['parking_lat'] = data['parking_lat']
flow_table['parking_lng'] = data['parking_lng']


polygons = set_polygons(100)[0]

path = os.getcwd() + '/유동인구_요일별/'

# mon = months[0]
for mon in months:
    print(mon,'시작')
    
    flow = pd.read_csv(path + 'WKDY_2018' + mon + '.csv', encoding = 'cp949')
    lat = flow['Y_COORD']
    lon = flow['X_COORD']

    values = {'MON':[],'TUS':[],'WED':[],'THU':[],'FRI':[],'SAT':[],'SUN':[]}
    
    for i in range(len(data)):
        values['MON'].append(0)
        values['TUS'].append(0)
        values['WED'].append(0)
        values['THU'].append(0)
        values['FRI'].append(0)
        values['SAT'].append(0)
        values['SUN'].append(0)

    for i in range(len(flow)):
        point = Point(lon[i],lat[i])
        
        for num in range(len(data)):
            polygon = polygons[num]
            
            if(polygon.contains(point)):
                
                for wd in weekday:
                    value = flow['FLOW_POP_CNT_' + wd][i]
                    values[wd][num] += value
                
    for wd in weekday:
        flow_table[mon +'_FLOW_POP_CNT_' + wd] = values[wd]

    print(mon,'완료')

flow_table.to_csv('WKDY_2018_2.csv',encoding = 'cp949')
print('전부 완료')

01 시작
01 완료
02 시작
02 완료
03 시작
03 완료
04 시작
04 완료
05 시작
05 완료
06 시작
06 완료
07 시작
07 완료
08 시작
08 완료
09 시작
09 완료
10 시작
10 완료
11 시작
11 완료
12 시작
12 완료
전부 완료


In [14]:
# ================================================================================
# 반경 100m 단속건수, cctv
# ================================================================================

polygons = set_polygons(100)[0]

illegal = pd.read_excel('단속건수.xlsx',encoding = 'cp949')

for i in range(len(values)):
    values[i] = 0

for i in range(len(illegal)):
    point = Point(illegal['lon'][i],illegal['lat'][i])
    
    for num in range(len(data)):
        polygon = polygons[num]
        if(polygon.contains(point)):
            values[num] += illegal['단속건수'][i]

data['D_100m'] = values

# data.to_csv('signature_table.csv', encoding = 'cp949')



illegal = pd.read_csv('cctv.csv',encoding = 'cp949')

for i in range(len(values)):
    values[i] = 0

for i in range(len(illegal)):
    point = Point(illegal['lon'][i],illegal['lat'][i])
    
    for num in range(len(data)):
        polygon = polygons[num]
        if(polygon.contains(point)):
            values[num] += 1

data['C_100m'] = values

data.to_csv('signature_table.csv', encoding = 'cp949')

In [15]:
# ================================================================================
# 반경 200m 단속건수, cctv
# ================================================================================

polygons = set_polygons(200)[0]

illegal = pd.read_excel('단속건수.xlsx',encoding = 'cp949')

for i in range(len(values)):
    values[i] = 0

for i in range(len(illegal)):
    point = Point(illegal['lon'][i],illegal['lat'][i])
    
    for num in range(len(data)):
        polygon = polygons[num]
        if(polygon.contains(point)):
            values[num] += illegal['단속건수'][i]

data['D_200m'] = values

# data.to_csv('signature_table.csv', encoding = 'cp949')


illegal = pd.read_csv('cctv.csv',encoding = 'cp949')

for i in range(len(values)):
    values[i] = 0

for i in range(len(illegal)):
    point = Point(illegal['lon'][i],illegal['lat'][i])
    
    for num in range(len(data)):
        polygon = polygons[num]
        if(polygon.contains(point)):
            values[num] += 1

data['C_200m'] = values

data.to_csv('signature_table.csv', encoding = 'cp949')

In [7]:
# ================================================================================
# CCTV 정제
# ================================================================================

polygons = set_polygons(200)[0]

data = pd.read_csv('signature_table_final.csv', encoding='cp949')
illegal = pd.read_csv('cctv_location.csv',encoding = 'cp949')

for i in range(len(values)):
    values[i] = 0

for i in range(len(illegal)):
    point = Point(illegal['경도'][i],illegal['위도'][i])
    value = illegal['대수'][i]
    
    for num in range(len(data)):
        polygon = polygons[num]
        if(polygon.contains(point)):
            values[num] += value

data['C'] = values

data.to_csv('signature_table_real_final.csv', encoding = 'cp949')

In [7]:
# ================================================================================
# 요일별 유동인구 테이블 만들기
# ================================================================================

import os
import pandas as pd

os.chdir('C:\\Users\\gkstm\\Desktop\\PIPA_data\\시그니쳐테이블')
path = os.getcwd() + '/유동인구_요일별/'

data = pd.read_csv('signature_table.csv', encoding = 'cp949')


months = [
    '01','02','03','04','05','06','07','08','09','10','11','12'
]

weekday = ['MON','TUS','WED','THU','FRI','SAT','SUN']



flow_table = pd.DataFrame()
flow_table['parking_name'] = data['parking_name']
flow_table['parking_lat'] = data['parking_lat']
flow_table['parking_lng'] = data['parking_lng']


polygons = set_polygons(100)[0]


# mon = months[0]
for mon in months:
    print(mon,'시작')
    
    flow = pd.read_csv(path + 'WKDY_2018' + mon + '.csv', encoding = 'cp949')
    lat = flow['Y_COORD']
    lon = flow['X_COORD']

    values = {'MON':[],'TUS':[],'WED':[],'THU':[],'FRI':[],'SAT':[],'SUN':[]}
    
    for i in range(len(data)):
        values['MON'].append(0)
        values['TUS'].append(0)
        values['WED'].append(0)
        values['THU'].append(0)
        values['FRI'].append(0)
        values['SAT'].append(0)
        values['SUN'].append(0)

    for i in range(len(flow)):
        point = Point(lon[i],lat[i])
        
        for num in range(len(data)):
            polygon = polygons[num]
            
            if(polygon.contains(point)):
                
                for wd in weekday:
                    value = flow['FLOW_POP_CNT_' + wd][i]
                    values[wd][num] += value
                
    for wd in weekday:
        flow_table[mon +'_FLOW_POP_CNT_' + wd] = values[wd]

    print(mon,'완료')

flow_table.to_csv('WKDY_2018.csv',encoding = 'cp949')
print('전부 완료')

01 시작
01 완료
02 시작
02 완료
03 시작
03 완료
04 시작
04 완료
05 시작
05 완료
06 시작
06 완료
07 시작
07 완료
08 시작
08 완료
09 시작
09 완료
10 시작
10 완료
11 시작
11 완료
12 시작
12 완료
전부 완료


In [11]:
# ================================================================================
# 시간별 유동인구 테이블 만들기
# ================================================================================

import os
import pandas as pd

os.chdir('C:\\Users\\gkstm\\Desktop\\PIPA_data\\시그니쳐테이블')
path = os.getcwd() + '/유동인구_시간별/'

data = pd.read_csv('signature_table.csv', encoding = 'cp949')


months = [
    '01','02','03','04','05','06','07','08','09','10','11','12'
]

times = [
    '00','01','02','03','04','05','06','07','08','09','10','11',
    '12','13','14','15','16','17','18','19','20','21','22','23'
]


flow_table = pd.DataFrame()
flow_table['parking_name'] = data['parking_name']
flow_table['parking_lat'] = data['parking_lat']
flow_table['parking_lng'] = data['parking_lng']


polygons = set_polygons(100)[0]


# mon = months[0]
for mon in months:
    print(mon,'시작')
    
    flow = pd.read_csv(path + 'TIME_2018' + mon + '.csv', encoding = 'cp949')
    lat = flow['Y_COORD']
    lon = flow['X_COORD']

    values = {}
    for t in times:
        values[t] = []
        for i in range(len(data)):
            values[t].append(0)

    for i in range(len(flow)):
        point = Point(lon[i],lat[i])
        
        for num in range(len(data)):
            polygon = polygons[num]
            
            if(polygon.contains(point)):
                
                for t in times:
                    value = flow['TMST_' + t][i]
                    values[t][num] += value
                
    for t in times:
        flow_table[mon +'_TMST_' + t] = values[t]

    print(mon,'완료')

flow_table.to_csv('TIME_2018.csv',encoding = 'cp949')
print('전부 완료')

01 시작
01 완료
02 시작
02 완료
03 시작
03 완료
04 시작
04 완료
05 시작
05 완료
06 시작
06 완료
07 시작
07 완료
08 시작
08 완료
09 시작
09 완료
10 시작
10 완료
11 시작
11 완료
12 시작
12 완료
전부 완료


In [9]:
import folium
m = folium.Map(location=[35.22323, 128.60946], zoom_start=11)

geometrys = set_polygons(200)[1]

for geo in geometrys:
    folium.GeoJson(
        geo,
        name='geojson',
        style_function=lambda feature: {
            'fillColor': 'white',
            'color' : 'black',
            'weight' : 1,
            'fillOpacity' : 0.2,
            }
    ).add_to(m)


    
    
m